In [10]:
import json
import os
import random
from tqdm import tqdm

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/synthetic-multiturn-multimodal/resolve/main/synthetic-multi-audio-relationship.jsonl
# !wget https://huggingface.co/datasets/mesolitica/synthetic-multiturn-multimodal/resolve/main/synthetic-multi-images-multi-audio-relationship.jsonl
# !wget https://huggingface.co/datasets/mesolitica/synthetic-multiturn-multimodal/resolve/main/synthetic-multi-images-relationship.jsonl

In [11]:
instructions_en = [
    'What is related between picture 1 and picture 2',
    'what is related for both pictures',
    'for both pictures, what is related'
]
instrunctions_ms = [
    'Apakah yang berkaitan antara gambar 1 dan gambar 2',
    'apa kaitan 2 gambar ni',
    'ada kaitan ke 2 gambar ni',
    'apa kaitan gambar-gambar tersebut'
]

In [12]:
en, ms = [], []
with open('synthetic-multi-images-relationship.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        filename = []
        for f in l['filename']:
            f = os.path.join('/home/ubuntu/filtered-blip-images', f)
            filename.append(f)
            
        if not all([os.path.exists(f) for f in filename]):
            continue
                
        en.append({
            'filename': filename,
            'conversations': [
                {'role': 'user', 'content': '<image> <image> ' + random.choice(instructions_en)},
                {'role': 'assistant', 'content': l['answer']},
            ]
        })
        ms.append({
            'filename': filename,
            'conversations': [
                {'role': 'user', 'content': '<image> <image> ' + random.choice(instrunctions_ms)},
                {'role': 'assistant', 'content': l['answer_ms']},
            ]
        })

100000it [00:02, 42884.74it/s]


In [14]:
a = set()
for l in en + ms:
    for i in range(len(l['conversations'])):
        splitted = l['conversations'][i]['content'].split('<')
        if len(splitted) > 1:
            a.add(splitted[1].strip().split('\n')[0])
a

{'image>'}

In [15]:
with open('prepared-relationship-en.jsonl', 'w') as fopen:
    for d in en:
        fopen.write(f'{json.dumps(d)}\n')

In [16]:
with open('prepared-relationship-ms.jsonl', 'w') as fopen:
    for d in ms:
        fopen.write(f'{json.dumps(d)}\n')